# Attack Password with Correlation Power Analysis IV.1 (CPA)

In [ ]:
%run '../util/Metadata.ipynb'
print_metadata()

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Improving-the-code" data-toc-modified-id="Improving-the-code-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Improving the code</a></span></li><li><span><a href="#Basic-Setup" data-toc-modified-id="Basic-Setup-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Basic Setup</a></span></li><li><span><a href="#Helper-Functions-for-Password-Attack" data-toc-modified-id="Helper-Functions-for-Password-Attack-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Helper Functions for Password Attack</a></span></li><li><span><a href="#New-Code---Old-results" data-toc-modified-id="New-Code---Old-results-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>New Code - Old results</a></span></li><li><span><a href="#Plotting-correlation-vs-traces" data-toc-modified-id="Plotting-correlation-vs-traces-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Plotting correlation vs traces</a></span></li><li><span><a href="#Hardening-against-CPA" data-toc-modified-id="Hardening-against-CPA-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Hardening against CPA</a></span><ul class="toc-item"><li><span><a href="#Random-start-point" data-toc-modified-id="Random-start-point-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Random start point</a></span></li><li><span><a href="#Dummy-operations" data-toc-modified-id="Dummy-operations-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>Dummy operations</a></span></li></ul></li><li><span><a href="#Summary" data-toc-modified-id="Summary-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Summary</a></span></li><li><span><a href="#Disconnect" data-toc-modified-id="Disconnect-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Disconnect</a></span></li></ul></div>

In this example we want to discuss possibilities to fight against a CPA attack.

## Improving the code

Let's first recap the password checking loop from `advanced-passwdcheck-xor`:
```c
passbad = 0;
for(uint8_t i = 0; i < sizeof(correct_passwd); i++){
    passbad |= correct_passwd[i] ^ passwd[i];
}
```

We revealed in the last example that aboves XOR generates a collision between known and determinable input data and secret data which shall be revealed by an attack.

So, how to lower this correlation?

## Basic Setup

Define Variables

In [ ]:
%run "../util/Init.ipynb"

Build target and upload

In [ ]:
TARGET = 'simpleserial-passwordcheck'
%store TARGET
%run "$HELPERSCRIPTS/Prepare.ipynb"

Import helper functions

In [ ]:
%run "$HELPERSCRIPTS/Setup_Generic.ipynb"

In [ ]:
scope.adc.samples = 500

## Helper Functions for Password Attack

In [ ]:
from bokeh.plotting import figure, show 
from bokeh.io import output_notebook
from bokeh.models import CrosshairTool, Label

output_notebook()

In [ ]:
import warnings
import random
import tqdm
import numpy as np

password_length = 8
"""Number of bytes of password"""

random_length = 32
"""Number of bytes of random input"""

def capture(command, data):
    scope.arm()
    target.simpleserial_write(command, data)

    ret = scope.capture()

    i = 0
    while not target.is_done():
        i += 1
        time.sleep(0.05)
        if i > 100:
            warnings.warn("Target did not finish operation")
            return None

    if ret:
        warnings.warn("Timeout happened during capture")
        return None

    return scope.get_last_trace()

def target_set_random(random_input=None):
    if random_input is not None:
        rand = random_input()
    else:
        rand = bytes(random.choices(range(0, 256), k=random_length))
    target.simpleserial_write('r', rand)
    return rand

def target_set_password(password):
    target.simpleserial_write('p', password)
    return target.simpleserial_read('r', password_length)

def target_check_password(command, password):
    target.simpleserial_write(command, password)
    return bytes(target.simpleserial_read('r', 1))[0] == 0

def capture_random(command, size=500, random_input=None, attempt_input=None):
    """Collect size number of password attempts with fully random random data."""
    traces = []
    textins = []
    rands = []
    for _ in tqdm.tqdm_notebook(range(size)):
        rands.append(target_set_random(random_input))
        if attempt_input is not None:
            pass_guess = attempt_input()
        else:
            pass_guess = bytes(random.choices(range(0, 256), k=password_length))
        traces.append(capture(command, pass_guess))
        textins.append(pass_guess)
    return np.array(traces), textins, rands

In [ ]:
import numpy as np
import warnings

warnings.simplefilter('ignore')

HW = [bin(n).count("1") for n in range(0, 256)]

def hw(n):
    return HW[n]

hw_vec = np.vectorize(hw)

def pearson(x: np.array, y: np.array):
    x_mean = np.mean(x)
    y_mean = np.mean(y)
    return sum((x - x_mean) * (y - y_mean)) / np.sqrt(sum((x - x_mean) ** 2) * sum((y - y_mean) ** 2))

def pearson_helper_traces(traces):
    return traces_diff, traces_squared

def pearson_pointwise(traces, intermediates):
    n = len(intermediates)
    d_traces = traces - np.einsum('ij->j', traces, dtype='float64', optimize='optimal') / np.double(n)
    d_intermediates = intermediates - np.einsum('i->', intermediates, dtype='float64', optimize='optimal') / np.double(n)
    
    tmp = np.einsum('ij,ij->j', d_traces, d_traces, optimize='optimal')
    tmp *= np.einsum('i,i->', d_intermediates, d_intermediates, optimize='optimal')

    return np.dot(d_intermediates, d_traces) / np.sqrt(tmp)

def pearson_pointwise_multi(traces, intermediates):
    (n, t) = traces.shape
    (_, m) = intermediates.shape

    d_traces = traces - np.einsum('nt->t', traces, dtype='float64', optimize='optimal') / np.double(n)
    d_intermediates = intermediates - np.einsum('nm->m', intermediates, dtype='float64', optimize='optimal') / np.double(n)
    
    tmp1 = np.einsum('nm,nm->m', d_intermediates, d_intermediates, optimize='optimal')
    tmp2 = np.einsum('nt,nt->t', d_traces, d_traces, optimize='optimal')
    tmp = np.einsum('m,t->mt', tmp1, tmp2, optimize='optimal')
    denominator = np.sqrt(tmp)
    numerator = np.einsum('nm,nt->mt', d_intermediates, d_traces, optimize='optimal')

    return np.nan_to_num(numerator / denominator)

In [ ]:
import math
import bokeh.palettes
import bokeh.transform
from bokeh.models import ColumnDataSource

def plot_correlation(correlations, color_palette=bokeh.palettes.Oranges6, **kw):
    kw['height'] = kw.get('height', 300)
    kw['y_range'] = kw.get('y_range', (-1, 1))
    p = figure(sizing_mode='stretch_width', **kw)
    p.vbar(
        x='points',
        top='corr',
        width=1,
        source=ColumnDataSource(data=dict(
            points=range(len(correlations)),
            corr=correlations,
            abscorr=abs(correlations),
        )),
        color=bokeh.transform.linear_cmap(
            field_name='abscorr', 
            palette=color_palette,
            low=1,
            high=0,
        ),
    )
    return p

def plot_correlation_vs_traces(
    traces,
    textins,
    password_index=0,
    trylist='abcdefghijklmnopqrstuvwxyz0123456789',
    plotpoints=500,
):
    # Compute data
    plotpoints = min(plotpoints, len(traces))
    data = np.zeros((len(trylist), plotpoints))
    intermediates = np.array([[hw(attempt[password_index] ^ ord(guess)) for guess in trylist] for attempt in textins])
    for i in range(0, plotpoints):
        j = math.ceil(i / plotpoints * len(traces))
        data[:, i] = np.max(np.abs(pearson_pointwise_multi(traces[:j, :], intermediates[:j, :])), axis=1)
    
    source = {
        'xs': len(data) * [list(range(0, len(traces), math.ceil(len(traces) / plotpoints)))],
        'ys': [corr for corr in data],
        'legend': list(trylist),
        'color': math.ceil(len(data) / 20) * bokeh.palettes.Category20_20,
    }
    # Create figure
    p = figure(sizing_mode='stretch_width', height=300, tooltips=[('char', '@legend'), ('corrleation', '$y')])
    p.multi_line(xs='xs', ys='ys', color='color', source=source)
    return p

## New Code - Old results

In [ ]:
password = b'ifx2019a'
target_set_password(password)
target_check_password('1', password)

In [ ]:
command = '1'

traces, textins, rands = capture_random(command=command, size=200)
correlations = pearson_pointwise(traces, np.array([hw(textin[0] ^ ord('i')) for textin in textins]))
print('max(correlations) = ', max(abs(correlations)))
show(plot_correlation(correlations))

We can observe that the aboves plot is exactly the same as in seen in the previous notebook. 

## Plotting correlation vs traces

In this example we took 200 traces and observed a high correlation coefficient. But, wouldn't last less traces?
Therefore we want to introduce an additional plot which gives an answer to this question. The **correlation vs. traces** plot. It can also be found in the examples of ChipWhisperer.

In [ ]:
show(plot_correlation_vs_traces(traces, textins))

With this graph it is easy to figure out that around 50 - 100 traces are enough to get a reliable result.

## Hardening against CPA

Let's recap the idea of the CPA: Somewhere in our code the piece of secret information (a character of the real password) *collides* with the some piece of information coming from extern (a character of the password attempt).

Can we avoid this collision? Not really. But we can *hide* it: Realize that it is not necessary that the comparison of the password characters is done in the order of their position inside the password. For instance you could do the whole comparison reversed but still getting the same result.

What would this mean for the CPA attack?

### Random start point

In the following investigations we will use command `2`. The code looks like this:
```c
uint8_t check_password_xor_randomstart(uint8_t *attempt)
{
    uint8_t passbad = 0;

    uint8_t j = random_buffer[0] % sizeof(password);

    trigger_high();
    for (uint8_t i = 0; i < sizeof(password); i++)
    {
        passbad |= password[j] ^ attempt[j];
        j = (j + 1) % sizeof(password);
    }
    trigger_low();

    simpleserial_put('r', 1, &passbad);

    return passbad;
}
```
`j` is used as index for the current character and its starting value is given by the `random_buffer`. Usually random is (of course) generated by the microcontroller itself. But in our case it is easier to feed in randomness from outside to have full control.

The following code block shows what happens if we give two possibilities for `j`. To achieve this, an explicit `random_function` is injected into the capturing process. This special function restricts the randomness of the first value to either 0 or 1. Thus, the loop starts either with the first or the second character.  

In [ ]:
def random_startpoint(startpoint):
    return lambda: bytes([random.randint(0, startpoint - 1)] + random.choices(range(256), k=random_length - 1))

command = '2'
target_check_password(command, password)

In [ ]:
traces, textins, rands = capture_random(command=command, size=200, random_input=random_startpoint(2))
show(plot_correlation_vs_traces(traces, textins))

We observe that the after 200 traces the maximal correlation is around 0.5. In contrast to the plain loop where it was around 0.9. This effect can be used more intensively if the increase the possible initial positions.

In [ ]:
traces, textins, rands = capture_random(command=command, size=200, random_input=random_startpoint(3))
show(plot_correlation_vs_traces(traces, textins))

If the first character has 3 possible indices it is impossible after 200 traces to achieve a reliable prediction. Is the loop secure now? Again: No! We just need more traces as the following plot shows.

In [ ]:
traces, textins, rands = capture_random(command=command, size=1000, random_input=random_startpoint(3))
show(plot_correlation_vs_traces(traces, textins))

In [ ]:
traces, textins, rands = capture_random(command=command, size=20000, random_input=random_startpoint(8))
show(plot_correlation_vs_traces(traces, textins))

If the starting point of the loop can use all possibilities out of the length of the password it is hard to achieve a reliable prediction after 20000 traces, but it is still possible.

### Dummy operations

So, do we need a longer password? Not necessarily. We just insert *dummy operations*. There are (in our case) 8 XOR operations which operate on the password and the attempt. By inserting a certain number of XOR operations which do not operate on the password or the attempt we could lower the probability of an index to proceed a certain position even more.

It is a common technique to hide real operations between a (high) number of dummy operations. There are some requirements on these dummy operations:
1. In a power trace it must be impossible to distinguish between dummy operations and real ones.
2. The computational result of the algorithm must stay the same.

In most cases it is quite hard to write down code with dummy operations satisfying the above conditions. For our password check there is an elegant way to incorporate dummy operations by just extending the buffers to be compared by some random values. 

```c
uint8_t check_password_xor_randomstart_random_buffer(uint8_t *attempt)
{
    uint8_t passbad = 0;
    uint8_t buffer1[sizeof(random_buffer)], buffer2[sizeof(random_buffer)];

    // Copy random input
    memcpy(buffer1, random_buffer, sizeof(random_buffer));
    memcpy(buffer2, random_buffer, sizeof(random_buffer));
    // Copy password and attempt
    memcpy(buffer1, password, sizeof(password));
    memcpy(buffer2, attempt, sizeof(password));

    uint8_t j = random_buffer[0] % sizeof(random_buffer);

    trigger_high();
    for (uint8_t i = 0; i < sizeof(random_buffer); i++)
    {
        passbad |= buffer1[j] ^ buffer2[j];
        j = (j + 1) % sizeof(random_buffer);
    }
    trigger_low();

    simpleserial_put('r', 1, &passbad);

    return passbad;
}
```

In [ ]:
command = '3'
target_check_password(command, password)

In [ ]:
traces, textins, rands = capture_random(command=command, size=20000)
show(plot_correlation_vs_traces(traces, textins))

Now every index has a probability of 1/32 to operate on a certain character. This lowers down the correlation so that even after 20000 traces it is not possible to give a reliable prediction.

These examples emphasize the main point of (side-channel) security: Usually nothing is safe against all attacks. It is just a matter of effort to break. And from a security software developers point of view it is an illusion to program anything absolute secure. The target is to achieve the *required amount of security*.

## Summary

In this notebook we observed:

* **Randomizing** the execution **order** of critical operations is an excellent way of lowering the strength of CPA.
* If a random order is not sufficient or not applicable (e.g. AES) it helps to hide the critical operations between a certain number of **dummy operations**.

## Disconnect

In [ ]:
scope.dis()
target.dis()